In [1]:
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct

# Initialize Clients
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["media_database"]
collection = db["raw_data"]

qdrant_client = QdrantClient("http://localhost:6333")
qdrant_client.recreate_collection(collection_name="media_embeddings", vector_size=768)

# Load Model
model = SentenceTransformer("all-MiniLM-L6-v2")

def featurize_and_store():
    for doc in collection.find():
        text = doc.get("content") or doc.get("title")
        embedding = model.encode(text).tolist()
        qdrant_client.upsert(
            collection_name="media_embeddings",
            points=[PointStruct(id=doc["_id"], vector=embedding, payload=doc)]
        )
        collection.update_one({"_id": doc["_id"]}, {"$set": {"embedding": embedding}})
    
if __name__ == "__main__":
    featurize_and_store()
    print("Featurization complete.")

ModuleNotFoundError: No module named 'sentence_transformers'